# 心理咨询论坛问答语料(无标签平面文本) - 制作迁徙学习数据集

为 NVIDIA/Megatron-LM 准备该领域的 finetune 数据集

## 说明

- 用问题与其多个答案中的一个反复组合，形成多个问题-答案对
- 使用 tag 或者问题中的关键字，选取一个类型的问答数据作为语料

来源语料格式举例：

```json
{'topics': ['婚姻'],
 'title': [],
 'text': ['其实无论我们遇见谁和谁相恋，无论成功与否都是过程和经历，即便是失败也不要沮丧，我们要回头在那失败的地方我们学到了什么，并且有什么美好的回忆留在心里，在人生道路上，只要我们认真对待感情，正正经经的恋爱，就算被辜负或者遇见错的人而失望，但我相信认真对待生活的人一定会有收获的，再困难再痛苦事情都会过去的，你依然是你，天也不会塌，经历会让我们成长，更懂得生活的意义，才会有精彩的人生。',
  '会有那么一个人会看见你的内在，拥抱你真挚的心。',
  '我说的对么?'],
 'answers': [['是的，', '做好自己，', '勇敢面对一切快乐和痛苦，', '一切源于我们的内心，', '修炼自己']]}
```

形成的平面带有分隔符的QA语料是：

```js
    plain_text(title) + "<sep>"
  + plain_text(text) + plain_text(slave_text) + "<sep>"
  + "<sep>"
  + "<|endoftext|>" + plain_text(answer)
```

将输出的语料存放在 JSON Lines 文件中，文本内容放在 `text` 属性中


## CD

切换到工作目录(**按实际情况，勿照搬下面的 Cell**)

In [1]:
%cd ..

/home/Public/Megatron-LM


## Importings

In [2]:
import os
import json
import random
from copy import copy
from contextlib import closing, ExitStack
import fileinput
from functools import partial
from glob import glob, iglob
from itertools import chain, repeat
from multiprocessing import Pool

import pandas as pd
import sentencepiece as spm
from tqdm.auto import tqdm, trange
from slugify import slugify

from IPython.display import display

## Tokenizer

使用与 Pretrained-Model 同样的 tokenizer

In [3]:
%%time

from data_utils.tokenization import SentencePieceTokenizer, make_tokenizer


TOKENIZER_MODEL_PATH='./data/spm/gpt2_huamei_corpus_bpe_32k_v2.model'

sp = spm.SentencePieceProcessor()
sp.load(TOKENIZER_MODEL_PATH)

print('SentencePiece 内置的特殊标记:')
for i in range(8):
  print(i, sp.id_to_piece(i), sp.is_control(i))
del sp
print()

print('Wrapped tokenizer ...')
tokenizer = make_tokenizer(
    SentencePieceTokenizer,
    None,
    model_path=TOKENIZER_MODEL_PATH
)

# tokenizer.command_tokens
for tok in tokenizer.command_name_map.values():
    print(f'{tok.Id}: {tok.token}')

# 需要的特殊标记：
SEP = '<sep>'
START_OF_ANSWER = '<|endoftext|>'

for piece in (SEP, START_OF_ANSWER):
    assert piece == tokenizer.IdToToken(tokenizer.TokenToId(piece))
    assert piece == tokenizer.IdToToken(tokenizer.EncodeAsIds(piece).tokenization[-1])


SentencePiece 内置的特殊标记:
0 <pad> True
1 <unk> False
2 <bos> True
3 <eos> True
4 <sep> False
5 <cls> False
6 <|endoftext|> False
7 一个 False

Wrapped tokenizer ...
0: <pad>
1: <eos>
2: <bos>
3: <unk>
4: <sep>
5: <L2R>
6: <ENC>
7: <MASK>
CPU times: user 892 ms, sys: 108 ms, total: 1 s
Wall time: 1.03 s


## Corpus

In [4]:
src_files = {
    'train': {
        'fp': '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/train_xinli_qax.json'
    },
    'val': {
        'fp': '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/valid_xinli_qax.json'   
    },
    'test': {
        'fp': '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/test_xinli_qax.json'  
    },
}

# 计算行数
for data in src_files.values():
    with open(data['fp']) as fp:
        data['lc'] = sum(1 for _ in tqdm(fp))

print('语料文件:')
display(src_files)


语料文件:


{'train': {'fp': '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/train_xinli_qax.json',
  'lc': 166941},
 'val': {'fp': '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/valid_xinli_qax.json',
  'lc': 9172},
 'test': {'fp': '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/test_xinli_qax.json',
  'lc': 7339}}

现在预览语料数据格式，随机选择一个：

In [5]:
n = random.randint(0, src_files['test']['lc'])

with open(src_files['test']['fp']) as fp:
    for i, line in tqdm(zip(range(n), fp), total=n):
        if i+1 < n:
            continue
        data = json.loads(line)
        display(i)
        display(data)


3809

{'topics': ['成长'],
 'title': ['如何改变自己的性格'],
 'text': ['我是一个很敏感，很懦弱，很胆小，很内向的人。',
  '这样的性格从小到大陪伴着我，从小学到高中，从大学到现在毕业半年，不曾有过什么变化。',
  '家里的亲戚对我的评价最多的就是“这孩子咋不吱声啊？',
  '这孩子怎么一句话不说啊？',
  '这孩子咋这么老实啊？”在学校的时候我就是这样一个不爱说话的人，跟班里的同学几乎没怎么说过话，没什么朋友，别人下课在聊天的时候而我一直都是一个人在那里坐着，我可以一天不说话，也没有人搭理我。',
  '那个时候我盼着上大学，我可以在大学里面改变自己。',
  '后来我上了大学，大学里有好多事情想做却都没有做，四年很快过去了，我还是没有什么变化，我还是这样。',
  '过年回家了，亲戚们在一起吃饭的时候还会说“这孩子怎么还是不说话啊？',
  '这孩子咋还不吱声啊？”一直到现在，我现在开始怀疑我是不是心理有问题，我到底应该怎么做才能改变我的性格？'],
 'answers': [['为何你觉得你有心理问题呢？', '为何你觉得你的性格需要改呢？'],
  ['恩，',
   '你这样哦，',
   '是不是和你小的时候成长环境有关呢？',
   '想一想自己为什么会这样呢，',
   '是因为害怕么，',
   '天性么，',
   '喜欢这样么？',
   '改变性格，',
   '是可以做到的，',
   '我之前也想你这样，',
   '那是因为我自卑，',
   '我害怕，',
   '我不敢与别人交谈，',
   '当然这跟我的成长环境有关，',
   '但是后来，',
   '我结交了一些朋友还有老师也倾听我的心事，',
   '一直在默默的帮我，',
   '要说我怎么转变的啊，',
   '是因为我爸爸去世了，',
   '恩，',
   '我就长大了一些，',
   '然后自我意识的不断完善，',
   '我就反抗的越来越强烈，',
   '要看个人意识吧，',
   '有一天我成功了，',
   '我不记得那是第几次对着妈妈又哭又吼，',
   '跟他说我这十多年来有多难过，',
   '她听了后，',
   '眼睛湿了，',
   '他对我说了一句“妈妈知道错了，',
   '我知道我之前对你的方

## 采样

### 采样条件

我们应：

- 选取问题+回答构成输入样本
- 一个问题可以对应多个回答
- 不要太短的
- 不要太长的
- 指定的主题

在此基础上，随机乱序选用若干样本用于今次的开发

- 主题

In [6]:
TOPIC = '婚姻'

- 长度限制定义

In [7]:
MIN_QUESTION_LENGTH = 16
MAX_QUESTION_LENGTH = 1024
MIN_ANSWER_LENGTH = 32
MAX_LENGTH = 2048

- 样本过滤函数

In [8]:
def extract_samples(line):
    line = line.strip()
    if not line:
        return []
    data = json.loads(line)
    result = []
#     # 主题不对的不要！
    topics = data.get('topics', []) 
    if TOPIC not in topics:
        return []
    # 没有回答的不要
    if not data.get('answers'):
        return []
    # 问题:
    title = ''.join(data['title']).strip()
    text = ''.join(data['text']).strip()
    # 既没有问题标题又没有问题内容的不要
    if not title and not text:
        return []
    n_title = len(tokenizer.EncodeAsIds(title))
    n_text = len(tokenizer.EncodeAsIds(text))
    # 太短不要
    if n_title + n_text < MIN_QUESTION_LENGTH:
        return []
    # 太长不要
    if n_title + n_text > MAX_QUESTION_LENGTH:
        return []
    # 回答：
    question_string = title + SEP + text + SEP
    # 答案
    for answer in data.get('answers', []):
        text = ''.join(answer).strip()
        length = len(tokenizer.EncodeAsIds(text))
        # 太短不要
        if length < MIN_ANSWER_LENGTH:
            continue
        # 太长不要
        if n_title + n_text + length > MAX_LENGTH:
            continue
        result.append(question_string + SEP + START_OF_ANSWER + text)
    #
    return result

### 进行采样

In [9]:
datasets = {}

for name, d in src_files.items():
    samples = datasets[name] = []
    with Pool() as pool, open(d['fp']) as fp:
        total = d['lc']
        mapper = pool.imap_unordered(
            extract_samples,
            tqdm(fp, total=total),
            chunksize=min(1024, total//os.cpu_count()+1)
        )
        for data_list in tqdm(mapper, total=total):
            samples.extend(data_list)

print(f'样本数量:')
for name, samples in datasets.items():
    print(f'\t{name}: {len(samples):,d}')



样本数量:
	train: 42,696
	val: 2,392
	test: 1,885


### 预览一条采样结果数据

In [10]:
s = random.choice(datasets['test'])
print(s)

<sep>我有轻微狐臭，手术后基本没有了。男友(一起两年，处俩月知我有腋臭)因此跟我分手，他说他能接受可他父母不能接受。我想知道大家对狐臭的看法，真的给另一半带来这么大负担吗？<sep><sep><|endoftext|>他能接受他父母不能接受，他是个孝子，姑娘，咱们成全他一辈子做孝子吧～～~我是毕业于宋仲基的故乡韩国思密达，愿以我的恰当对待，温暖陪伴，换取你敢于挣脱过去束缚的勇气。欢迎点我头像关注我，或私信我交流更多，也可直接预约我，我会主动联系您。


## 保存结果

### 输出目录

In [11]:
OUTPUT_DIR = os.path.abspath(
    os.path.join(
        'data',
        'xinliqa-{0}'.format(slugify(TOPIC, separator='')),
    )
)

print(f'输出目录: {OUTPUT_DIR}')
os.makedirs(OUTPUT_DIR, exist_ok=True)

输出目录: /home/Public/Megatron-LM/data/xinliqa-hunyin


乱序后直接保存所有输出样本到文件

In [12]:
for name, samples in datasets.items():
    fname = os.path.join(OUTPUT_DIR, f'{name}.json')
    with open(fname, 'w') as fp:
        random.shuffle(samples)
        for txt in tqdm(samples, name):
            print(json.dumps({'text': txt}, ensure_ascii=False), file=fp)


检查文件:

In [13]:
!echo "size:"
!du -hc {OUTPUT_DIR}/*.json

!echo

!echo "line:"
!wc -l {OUTPUT_DIR}/*.json

size:
2.2M	/home/Public/Megatron-LM/data/xinliqa-hunyin/test.json
48M	/home/Public/Megatron-LM/data/xinliqa-hunyin/train.json
2.7M	/home/Public/Megatron-LM/data/xinliqa-hunyin/val.json
53M	总用量

line:
    1885 /home/Public/Megatron-LM/data/xinliqa-hunyin/test.json
   42696 /home/Public/Megatron-LM/data/xinliqa-hunyin/train.json
    2392 /home/Public/Megatron-LM/data/xinliqa-hunyin/val.json
   46973 总用量


## 测试数据加载

In [42]:
with fileinput.input(iglob(f'{OUTPUT_DIR}/*.json')) as fp:
    for line in fp:
        data = json.loads(line)
        text = data['text']
        print(text)
        break

<sep>不知道还要不要坚持 男朋友对我时好时坏的 有时候说的话很难听让我很想分手但是又说不出口让我很纠结 对我好的时候很好 对我不好的时候会打你狠话<sep><sep><|endoftext|>要在心理留有一个底线。不管他好的时候多好，坏的时候碰到了底线，你就该果断点。不然一辈子都只能在纠结中度过。
